##Installing Pytorch##

In [ ]:
!pip3 install torch

##Importing##

In [ ]:
import torch
import numpy as np

In [ ]:
torch.__version__

In [ ]:
lst = [1,2,3,4]

In [ ]:
lst

In [ ]:
arr = np.array(lst)

In [ ]:
arr

In [ ]:
arr.dtype

**Convert numpy to tensor**

In [ ]:
tensors = torch.from_numpy(arr)

In [ ]:
tensors[-1:3]

In [ ]:
tensor_arr = torch.tensor(arr)

In [ ]:
tensor_arr

In [ ]:
tensor_arr[2]=30
print(tensor_arr)
print(arr)

In [ ]:
zeros=torch.zeros(2,3,dtype=torch.int64)
ones=torch.ones(2,3,dtype=torch.int64)
print(zeros)
print(ones)

In [ ]:
a = torch.tensor([2,3,4], dtype=torch.float)
b = torch.tensor([5,6,7], dtype=torch.float)

In [ ]:
a+b

In [ ]:
torch.add(a,b)

In [ ]:
c=torch.zeros(3)

In [ ]:
torch.add(a,b,out=c)

In [ ]:
c

In [ ]:
c.sum()

In [ ]:
k=a.mul(b)
l=a.dot(b)
print(k)
print(l)

##Back propagation using Pytorch

In [ ]:
import torch

In [ ]:
x = torch.tensor(5,requires_grad=True, dtype = torch.float)
y=x**2
y.backward()

In [ ]:
y

In [ ]:
x.grad

In [ ]:
lst1 = [[1,2,3],[4,5,6],[7,8,9]]
tensor_input = torch.tensor(lst1,dtype=torch.float,requires_grad=True)

In [ ]:
tensor_input

In [ ]:
y=tensor_input**2 + tensor_input**3 

In [ ]:
y

In [ ]:
z= y.sum()

In [ ]:
z

In [ ]:
z.backward()

In [ ]:
tensor_input.grad

In [ ]:
y

##ANN in Pytorch

**Load Kaggle Dataset directly**

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download uciml/pima-indians-diabetes-database

In [ ]:
! unzip pima-indians-diabetes-database

**Code**

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
X=df.drop('Outcome',axis=1).values
y=df['Outcome'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

**Pytorch Model**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
X_train = torch.tensor(X_train, dtype = torch.float)
X_test = torch.tensor(X_test, dtype = torch.float)

In [ ]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class ANN_Model(nn.Module):
  def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
    super().__init__()
    self.f_connected1 = nn.Linear(input_features,hidden1)
    self.f_connected2 = nn.Linear(hidden1,hidden2)
    self.out = nn.Linear(hidden2,out_features)
  def forward(self,x):
    x=F.relu(self.f_connected1(x))
    x=F.relu(self.f_connected2(x))
    x=self.out(x)
    return x

In [ ]:
torch.manual_seed(20)
model=ANN_Model()

In [ ]:
model.parameters

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
epochs=500
final_loss=[]
for i in range(epochs):
  i=i+1
  y_pred=model.forward(X_train)
  loss=loss_function(y_pred,y_train)
  final_loss.append(loss)
  if i%10==0:
    print("Epoch number: {} and the loss: {}".format(i,loss.item()))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


In [ ]:
predictions=[]
with torch.no_grad():
  for i,data in enumerate(X_test):
    y_pred=model(data)
    predictions.append(y_pred.argmax().item())
    # print(y_pred,y_pred.argmax().item())

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predictions)
cm

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)

In [ ]:
score

##ASSIGNMENT-2

Learn how to use CNNs: train from scratch, finetune a pretrained model, use a pre-trained model as it is.


##Imports

In [1]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 17.3 MB/s 
     |████████████████████████████████| 47.8 MB 2.3 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [2]:
#for import albumentations as A
!pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 21.6 MB 60.2 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.64
    Uninstalling opencv-python-headless-4.5.5.64:
      Successfully uninstalled opencv-python-headless-4.5.5.64


In [3]:
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F


from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
import numpy as np
import random

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

from itertools import chain

##Enabling GPU

In [ ]:
#Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(torch.cuda.get_device_name(0))

##Download iNaturalist-12K dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
actual_data_path = "/content/drive/MyDrive/inaturalist_12K"

Mounted at /content/drive/


In [5]:
#######################################################
#               Define Transforms
#######################################################

#To define an augmentation pipeline, you need to create an instance of the Compose class.
#As an argument to the Compose class, you need to pass a list of augmentations you want to apply. 
#A call to Compose will return a transform function that will perform image augmentation.
#(https://albumentations.ai/docs/getting_started/image_augmentation/)

train_transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=350),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=360, p=0.5),
        A.RandomCrop(height=256, width=256),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.MultiplicativeNoise(multiplier=[0.5,2], per_channel=True, p=0.2),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        ToTensorV2(),
    ]
)

test_transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=350),
        A.CenterCrop(height=256, width=256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [6]:
####################################################
#       Create Train, Valid and Test sets
####################################################
train_data_path = os.path.join(actual_data_path, "train")
test_data_path = os.path.join(actual_data_path, "val")

train_image_paths = [] #to store image paths in list
classes = [] #to store class values

#1.
# get all the paths from train_data_path and append image paths and class to to respective lists
# eg. train path-> 'images/train/26.Pont_du_Gard/4321ee6695c23c7b.jpg'
# eg. class -> 26.Pont_du_Gard
for data_path in glob.glob(train_data_path + "/*"):
  classes.append(data_path.split('/')[-1]) 
  train_image_paths.append(glob.glob(data_path + '/*'))
train_image_paths = list(chain.from_iterable(train_image_paths))
random.shuffle(train_image_paths)

print(len(train_image_paths))
print('train_image_path example: ', train_image_paths[2])
print('class example: ', classes[2])

#2.
# split train valid from train paths (80,20)
train_image_paths, valid_image_paths = train_image_paths[:int(0.9*len(train_image_paths))], train_image_paths[int(0.9*len(train_image_paths)):] 

#3.
# create the test_image_paths
test_image_paths = []
for data_path in glob.glob(test_data_path + '/*'):
    test_image_paths.append(glob.glob(data_path + '/*'))

test_image_paths = list(chain.from_iterable(test_image_paths))

print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_image_paths), len(valid_image_paths), len(test_image_paths)))


10042
train_image_path example:  /content/drive/MyDrive/inaturalist_12K/train/Reptilia/95261cb10b22f1d91d129fe85728cd09.jpg
class example:  Fungi
Train size: 9037
Valid size: 1005
Test size: 2000


In [7]:
#######################################################
#      Create dictionary for class indexes
#######################################################

idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

In [8]:
class_to_idx

{'Amphibia': 1,
 'Animalia': 3,
 'Arachnida': 0,
 'Aves': 7,
 'Fungi': 2,
 'Insecta': 8,
 'Mammalia': 5,
 'Mollusca': 4,
 'Plantae': 6,
 'Reptilia': 9}

In [9]:
#######################################################
#               Define Dataset Class
#######################################################

class iNaturalist_12KDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label
    


In [10]:
#######################################################
#                  Create Dataset
#######################################################

train_dataset = iNaturalist_12KDataset(train_image_paths,train_transforms)
valid_dataset = iNaturalist_12KDataset(valid_image_paths,test_transforms) #test transforms are applied
test_dataset = iNaturalist_12KDataset(test_image_paths,test_transforms)

In [11]:
print(train_dataset.__len__())
# for i in range(train_dataset.__len__()):
#   print('The shape of tensor for 50th image in train dataset: ',train_dataset[i][0].shape)
#   print('The label for 50th image in train dataset: ',train_dataset[i][1])

9037


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#######################################################
#                  Visualize Dataset
#         Images are plotted after augmentation
#######################################################

def visualize_augmentations(dataset, idx=0, samples=10, cols=5, random_img = False):
    
    dataset = copy.deepcopy(dataset)
    #we remove the normalize and tensor conversion from our augmentation pipeline
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    
        
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 8))
    for i in range(samples):
        if random_img:
            idx = np.random.randint(1,len(train_image_paths))
        image, lab = dataset[idx]
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
        ax.ravel()[i].set_title(idx_to_class[lab])
    plt.tight_layout(pad=1)
    plt.show()    

visualize_augmentations(train_dataset,np.random.randint(1,len(train_image_paths)), random_img = True)


In [12]:
#######################################################
#                  Define Dataloaders
#######################################################

train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True
)

valid_loader = DataLoader(
    valid_dataset, batch_size=64, shuffle=True
)


test_loader = DataLoader(
    test_dataset, batch_size=64, shuffle=False
)


## Building the Model

In [21]:
class CnnModel(nn.Module):
  def __init__(self, conv_attributes, pool_attributes,in_feature):
    # print("PRITHAJ 0---------------")
    super(CnnModel, self).__init__()
    self.conv = []
    self.pool = []
    for i in range(5):
      self.conv.append(nn.Conv2d(conv_attributes[i]["in_channels"], conv_attributes[i]["out_channels"], conv_attributes[i]["kernel_size"]))
      self.pool.append(nn.MaxPool2d(pool_attributes[i]["kernel_size"], pool_attributes[i]["stride"]))
    self.fc1 = nn.Linear(in_feature, 10)

    # self.conv1 = nn.Conv2d(conv_attributes[0]["in_channels"], conv_attributes[0]["out_channels"], conv_attributes[0]["kernel_size"])
    # # print("conv1 shape",self.conv1.size)
    # self.pool = nn.MaxPool2d(2, 2)
    # # print(self.pool.shape)
    # self.conv2 = nn.Conv2d(conv_attributes[1]["in_channels"], conv_attributes[1]["out_channels"], conv_attributes[1]["kernel_size"])
    # # print("conv2 shape",self.conv2.size)
    # self.conv3 = nn.Conv2d(conv_attributes[2]["in_channels"], conv_attributes[2]["out_channels"], conv_attributes[2]["kernel_size"])
    # # print("conv3 shape",self.conv2.size)
    # self.conv4 = nn.Conv2d(conv_attributes[3]["in_channels"], conv_attributes[3]["out_channels"], conv_attributes[3]["kernel_size"])
    # # print("conv4 shape",self.conv2.size)
    # self.conv5 = nn.Conv2d(conv_attributes[4]["in_channels"], conv_attributes[4]["out_channels"], conv_attributes[4]["kernel_size"])
    # # print("conv5 shape",self.conv2.size)
    # input = self.conv5.view(64, -1) # torch.Size([1, 784])
    # print("input shape",input.shape)
    # print(self.fc1.shape)

  def forward(self,x):
    # -> n, 3, 32, 32
    # print("PRITHAJ---------------")
    # print(x.shape)
    for i in range(5):
      x = self.pool[i](F.relu(self.conv[i](x)))
    x = torch.flatten(x, 1) # flatten all dimensions except batch
    x = self.fc1(x)                       # -> n, 10

    # x = self.pool(F.relu(self.conv(x)))  # -> n, 6, 14, 14
    # # print("1",x.shape)
    # x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
    # # print("2",x.shape)
    # x = self.pool(F.relu(self.conv3(x)))  # -> n, 16, 5, 5
    # # print("3",x.shape)
    # x = self.pool(F.relu(self.conv4(x)))  # -> n, 16, 5, 5
    # # print("4",x.shape)
    # x = self.pool(F.relu(self.conv5(x)))  # -> n, 16, 5, 5
    # # print("5",x.shape)
    # print("dense shape",x.shape)
    # x = x.view(-1, 16 * 61 * 61)            # -> n, 400
    return x

In [ ]:
def TrainNetwork(model,num_epochs, batch_size,learning_rate):
  # #model = CNN_Model().to(Device)
  # model = CnnModel()
  # print(model)
  loss_funt = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

  n_total_steps = len(train_loader)
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
          #print(images)
          #print(labels)
          # origin shape: [4, 3, 32, 32] = 4, 3, 1024
          # input_layer: 3 input channels, 6 output channels, 5 kernel size
          #images = images.to(Device)
          #labels = labels.to(Device)

          # Forward pass
          # print(i,images.shape,labels.shape)
          outputs = model(images)
          loss = loss_funt(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i+1) % 20 == 0:
              print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

  print('Finished Training')
  # PATH = './cnn.pth'
  # torch.save(model.state_dict(), PATH)



In [ ]:
def TestNetwork(model,num_epochs, batch_size,learning_rate):
  with torch.no_grad():
      n_correct = 0
      n_samples = 0
      n_class_correct = [0 for i in range(10)]
      # print("n_class_correct", n_class_correct)
      n_class_samples = [0 for i in range(10)]
      # print("n_class_correct",n_class_correct)
      for images, labels in test_loader:
          #images = images.to(Device)
          #labels = labels.to(Device)
          # print("images.shape",images.shape)
          # print("labels.shape",labels.shape)
          outputs = model(images)
          # print("outputs",outputs)
          # max returns (value ,index)
          _, predicted = torch.max(outputs, 1)
          # print("predicted",predicted)
          n_samples += labels.size(0)
          n_correct += (predicted == labels).sum().item()
          # print("n_samples",n_samples)
          # print("n_correct",n_correct)
          # print("PREDICTED SIZE",predicted.size()[0])
          for i in range(predicted.size()[0]):
              label = labels[i]
              # print("label",label)
              pred = predicted[i]
              # print("pred",pred)
              if (label == pred):
                  n_class_correct[label] += 1
                  # print("n_class_correct",n_class_correct)
              n_class_samples[label] += 1
              # print("n_class_samples",n_class_samples)

      acc = 100.0 * n_correct / n_samples
      print(f'Accuracy of the network: {acc} %')

      for i in range(10):
          acc = 100.0 * n_class_correct[i] / n_class_samples[i]
          print(f'Accuracy of {classes[i]}: {acc} %')

In [34]:
##Calculates the input feature for the dense linear layer
def LinearInFeatureCalculate(initial_dim,conv_attributes,pool_attributes):
  for i in range(5):
    D = (initial_dim + 2*conv_attributes[i]["padding"] - conv_attributes[i]["dilation"]*(conv_attributes[i]["kernel_size"]-1) - 1)//(conv_attributes[i]["stride"]) + 1
    D = D//pool_attributes[i]["stride"]
    initial_dim = D
  return D


**Main function**

In [37]:
def main():
  print("Hello")
  conv_attributes = [{"in_channels":0,"out_channels":0,"kernel_size":0, "stride":1, "padding":0, "dilation":1},
                     {"in_channels":0,"out_channels":0,"kernel_size":0, "stride":1, "padding":0, "dilation":1},
                     {"in_channels":0,"out_channels":0,"kernel_size":0, "stride":1, "padding":0, "dilation":1},
                     {"in_channels":0,"out_channels":0,"kernel_size":0, "stride":1, "padding":0, "dilation":1},
                     {"in_channels":0,"out_channels":0,"kernel_size":0, "stride":1, "padding":0, "dilation":1}]
  
  
  ##Attributes for 1st Convolution Layer
  conv_attributes[0]["in_channels"]=3
  conv_attributes[0]["out_channels"]=6
  conv_attributes[0]["kernel_size"]=3

  ##Attributes for 2nd Convolution Layer
  conv_attributes[1]["in_channels"]=6
  conv_attributes[1]["out_channels"]=12
  conv_attributes[1]["kernel_size"]=3

  ##Attributes for 3rd Convolution Layer
  conv_attributes[2]["in_channels"]=12
  conv_attributes[2]["out_channels"]=16
  conv_attributes[2]["kernel_size"]=5

  ##Attributes for 4th Convolution Layer
  conv_attributes[3]["in_channels"]=16
  conv_attributes[3]["out_channels"]=32
  conv_attributes[3]["kernel_size"]=5

  ##Attributes for 5th Convolution Layer
  conv_attributes[4]["in_channels"]=32
  conv_attributes[4]["out_channels"]=64
  conv_attributes[4]["kernel_size"]=7

  pool_attributes = [{"kernel_size":1, "stride": 1},
                     {"kernel_size":1, "stride": 1},
                     {"kernel_size":1, "stride": 1},
                     {"kernel_size":1, "stride": 1},
                     {"kernel_size":1, "stride": 1}]

  pool_attributes[0]["kernel_size"]=2
  pool_attributes[0]["stride"]=2

  pool_attributes[1]["kernel_size"]=2
  pool_attributes[1]["stride"]=2
  
  pool_attributes[2]["kernel_size"]=2
  pool_attributes[2]["stride"]=2

  pool_attributes[3]["kernel_size"]=2
  pool_attributes[3]["stride"]=2

  pool_attributes[4]["kernel_size"]=2
  pool_attributes[4]["stride"]=2

  final_dim=LinearInFeatureCalculate(256,conv_attributes,pool_attributes)

  in_feature = (final_dim ** 2) * conv_attributes[4]["out_channels"]
  print(in_feature)

  #model = CNN_Model().to(Device)
  model = CnnModel(conv_attributes, pool_attributes,in_feature)

In [38]:
if  __name__ =="__main__":
  main()

Hello
576


NameError: ignored